In [1]:
from IPython.display import clear_output

import re
import time
import datetime

import requests
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

from tinydb import TinyDB, Query

In [2]:
db = TinyDB('data/custom_data/oddsportal_output.json')

In [3]:
tournaments = db.table('tournaments')
matches = db.table('matches')

In [4]:
query = Query()

In [5]:
"""
UPDATE THESE
"""
username = "simi2525"
password = "u7YZbkpPsyGg3pE"
chrome_executable_path = r"C:\Users\Cristian Simionescu\Desktop\Master\Random\webdrivers\chromedriver.exe"
output_file_path = r"data/custom_data/oddsportal_output.txt"

In [10]:
base_url = "https://www.oddsportal.com"
start_url = "https://www.oddsportal.com/tennis/results/"
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Mobile Safari/537.36"
}

options = Options()
# options.add_argument("--headless") # Runs Chrome in headless mode.
# options.add_argument('--no-sandbox') # Bypass OS security model
# options.add_argument('--disable-gpu')  # applicable to windows os only
# options.add_argument('start-maximized') # 
# options.add_argument('disable-infobars')
# options.add_argument("--disable-extensions")
# options.headless = True

In [11]:
def login(username,password,c):
    c.get("https://www.oddsportal.com/login/")
    username_box = c.find_element_by_name("login-username")
    password_box = c.find_element_by_name("login-password")
    time.sleep(2)
    username_box.send_keys(username)
    time.sleep(2)
    password_box.send_keys(password)
    username_box.send_keys(Keys.ENTER)
    time.sleep(5)

In [13]:
with requests.get(start_url, headers=headers) as r:
    links = re.findall(b'foo="f"\s*href="([^"]+)', r.content)
links = list(map(lambda x: x.decode(), links))

c = Chrome(executable_path=chrome_executable_path,options=options)
c.get("https://www.oddsportal.com/login/")

T_START_INDEX = 89
Y_START_INDEX = 0
M_START_INDEX = 3

T_END_INDEX = len(links)
Y_END_INDEX = None    
M_END_INDEX = None

for index, link in enumerate(links[T_START_INDEX:T_END_INDEX]):
    full_link = base_url + link
    link = link.lstrip('/').replace('tennis/', '')
    sep_index = link.find('/')
    if sep_index < 0:
        continue  # not found

    country = link[:sep_index]
    
    c.get(full_link)
    if "Page not found" in c.title:
        continue
    tournament_name = c.find_element_by_xpath('//h1').get_attribute('textContent')
    surface = re.search('\(\s*([a-z]+)\s*\)', tournament_name)
    sex = "men"
    if "women" in tournament_name.lower() or "wta" in tournament_name.lower():
        sex = "women"
        #We only want ATP
        continue
    elif "mixed" in tournament_name.lower():
        sex = "mixed"
        #We only want ATP
        continue
        
    doubles = "doubles" in tournament_name.lower()
    if doubles:
        continue
        
#     header = c.find_element_by_id("user-header").get_attribute('textContent')
#     if not username in header:
#         print("Not loged in")
#         login(username,password,c)
    
    if surface:
        surface = surface.group(1)
    else:
        surface = ""
    tournament_name = tournament_name.split(surface)[0].replace("(","").strip()
    
    tournmanet_year_links = c.find_element_by_css_selector("#col-content > div.main-menu2.main-menu-gray > ul.main-filter").find_elements_by_css_selector("li > span > strong > a")
    tournmanet_year_links = [e.get_attribute("href") for e in tournmanet_year_links]
    
    if Y_END_INDEX is None:
        Y_END_INDEX = len(tournmanet_year_links)
    for year_index,tournmanet_year_link in enumerate(tournmanet_year_links[Y_START_INDEX:Y_END_INDEX]):
        c.get(tournmanet_year_link)

        tournament_year = c.find_element_by_css_selector("#col-content > div.main-menu2.main-menu-gray > ul.main-filter > li > span.active").get_attribute("textContent").strip()

        try:
            prize_money = c.find_element_by_css_selector("#col-content > div.prizemoney").get_attribute("textContent").replace("Prize money: ","").strip()
        except:
            prize_money = "$0"
        
        tournament_id = tournament_name+" - "+tournament_year
        tournaments.remove(query.id==tournament_id)        
        tournaments.insert({"id":tournament_id,"year":int(tournament_year),"doubles":doubles,"prize_money":prize_money,"surface":surface,"sex":sex})
        
        table = c.find_element_by_id('tournamentTable')
        table = table.find_element_by_xpath('.//tbody')
        
        match_links = []
        for child in table.find_elements_by_xpath('.//tr'):
            if child.get_attribute('class') == "dark center":
                pass
            elif "nob-border" in child.get_attribute('class'):
                pass
            elif 'deactivate' in child.get_attribute('class'):
                match_links.append(child.find_element_by_css_selector('td > a').get_attribute("href"))
        
        if M_END_INDEX is None:
            M_END_INDEX = len(match_links)
        for match_id, match_link in enumerate(match_links[M_START_INDEX:M_END_INDEX]): 
            clear_output(wait=True)
            print("Tournament ID",index+T_START_INDEX)
            print("Tournament year ID",year_index+Y_START_INDEX)
            print("Match ID",match_id+M_START_INDEX)
            print("Tournaments remaining",T_END_INDEX-T_START_INDEX-index)
            print("Tournament years remaining",len(tournmanet_year_links)-Y_START_INDEX-year_index)
            print("Matches in tournament remainig",len(match_links)-M_START_INDEX-match_id)
            
            print(tournament_id)

            match_time = players = final_score = info_val = ""
            odds = list()

            c.get(match_link)

            players = c.find_element_by_css_selector("#col-content > h1").get_attribute("textContent")
            players = re.sub('<[^>]+>', '', players).split('-')

            match_time = c.find_element_by_css_selector("#col-content > p.date.datet").get_attribute("textContent")
            match_time = datetime.datetime.strptime(match_time.replace("  "," "),'%A, %d %b %Y, %H:%M')

            result = c.find_element_by_css_selector("#event-status > p").get_attribute("textContent")
            if "Final result" not in result:
                continue
            result = result.replace("Final result","").strip()
            result = result.split(" ",1)
            score = result[0].strip()
            if len(result) == 2:
                sets = result[1][1:-1].split(", ")
            else:
                sets = []
            try:
                exists = c.find_element_by_css_selector("#odds-data-table > div.table-container > table.table-main.detail-odds.sortable > tfoot > tr.odd > td > a")
            except:
                exists = False
            
            if exists != False and "Click to show " in exists.get_attribute("textContent") and "Click to show 0 more bookmakers!" not in exists.get_attribute("textContent"):
                exists.click()
            time.sleep(0.5)
            odds_table = c.find_element_by_css_selector("#odds-data-table > div.table-container > table.table-main.detail-odds.sortable")
            odds_body = odds_table.find_element_by_css_selector("tbody")
            if "Log in to display the odds!" in odds_table.find_element_by_css_selector("tfoot").get_attribute("textContent"):
                print("Not loged in")
                login(username,password,c)
                c.get(match_link)
                time.sleep(2)
                odds_table = c.find_element_by_css_selector("#odds-data-table > div.table-container > table.table-main.detail-odds.sortable")
                odds_body = odds_table.find_element_by_css_selector("tbody")
            
            odds = {}
            for odd in odds_body.find_elements_by_css_selector("tr.lo"):
                columns = odd.find_elements_by_css_selector("td")
                if len(columns) < 3:
                    continue
                bookie = columns[0].get_attribute("textContent").strip()
                rates = [columns[1].get_attribute("textContent").strip(),columns[2].get_attribute("textContent").strip()]
                odds[bookie] = rates
            matches.remove(query.tournament_id==tournament_id and query.match_time==match_time)        
            matches.insert({"tournament_id":tournament_id,"country":country,"surface":surface,"match_time":str(match_time),"players":players,"score":score,"sets":sets,"odds":odds})
    Y_START_INDEX = 0
    M_START_INDEX = 0
    Y_END_INDEX = None    
    M_END_INDEX = None
c.close()

Tournament ID 89
Tournament year ID 0
Match ID 3
Tournaments remaining 2501
Tournament years remaining 11
Matches in tournament remainig 5
Exhibition Kooyong Men - 2019


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#odds-data-table > div.table-container > table.table-main.detail-odds.sortable"}
  (Session info: chrome=75.0.3770.100)


In [9]:
# matches.purge()
# tournaments.purge()